In [1]:
# import cv2
# import serial
# import time
# import numpy as np

# # Initialize serial communication with Arduino
# arduino = serial.Serial('COM5', 700000, timeout=2)  # Timeout after 2 seconds
# time.sleep(2)  # Wait for the connection to be established

# # Load pre-trained SSD MobileNet model
# config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
# frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# # Initialize the detection model
# model = cv2.dnn_DetectionModel(frozen_model, config_file)

# # Load class labels
# classLabels = []
# file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
# with open(file_name, 'rt') as fpt:
#     classLabels = fpt.read().rstrip('\n').split('\n')

# # Set model parameters
# model.setInputSize(320, 320)
# model.setInputScale(1.0 / 127.5)
# model.setInputMean((127.5, 127.5, 127.5))
# model.setInputSwapRB(True)

# # Define font and font scale for displaying labels
# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 0.5

# # Image dimensions from the Arduino (320x240 for QVGA)
# image_width = 320
# image_height = 240

# # Function to convert RGB565 to RGB888
# def rgb565_to_rgb888(data):
#     r = (data >> 11) & 0x1F
#     g = (data >> 5) & 0x3F
#     b = data & 0x1F
#     r = (r << 3) & 0xFF  # Scale 5-bit red to 8-bit
#     g = (g << 2) & 0xFF  # Scale 6-bit green to 8-bit
#     b = (b << 3) & 0xFF  # Scale 5-bit blue to 8-bit
#     return r, g, b

# # Read the image data from Arduino and process it
# def read_image_from_arduino():
#     image_data = []
#     start_time = time.time()
#     bytes_per_image = image_width * image_height * 2  # 2 bytes per pixel (RGB565)

#     while len(image_data) < bytes_per_image:
#         if arduino.in_waiting > 0:
#             byte = arduino.read(1)
#             image_data.append(ord(byte))
#         elif time.time() - start_time > 2:  # Timeout after 2 seconds
#             print("Timeout: No data received from Arduino")
#             return None  # Return None if no data was received
    
#     # Convert the raw data into RGB565 format
#     image_rgb565 = np.array(image_data, dtype=np.uint8).reshape((image_height, image_width, 2))
    
#     # Convert RGB565 to RGB888 for OpenCV
#     image_rgb = np.zeros((image_height, image_width, 3), dtype=np.uint8)
#     for y in range(image_height):
#         for x in range(image_width):
#             pixel_data = (image_rgb565[y, x, 0] << 8) | image_rgb565[y, x, 1]
#             r, g, b = rgb565_to_rgb888(pixel_data)
#             image_rgb[y, x] = [b, g, r]  # OpenCV uses BGR format

#     # Resize the image to the input size expected by the model
#     image_resized = cv2.resize(image_rgb, (320, 240))

#     return image_resized

# # Main loop for detection
# while True:
#     # Read the image from the Arduino
#     frame = read_image_from_arduino()
    
#     if frame is None:
#         continue  # Skip this loop iteration if no frame is received

#     # Detect objects in the frame
#     classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

#     # Check if any objects are detected
#     if len(classIndex) != 0:
#         for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
#             if ClassInd <= 80:  # Process only known COCO classes (1-80)
#                 # Draw bounding box around detected objects
#                 cv2.rectangle(frame, boxes, (0, 255, 0), 2)
#                 cv2.putText(frame, classLabels[ClassInd - 1], 
#                             (boxes[0], boxes[1] - 10), font, 
#                             fontScale=font_scale, color=(0, 255, 0), thickness=2)
                
#                 # Activate Arduino buzzer for 'car', vibrator for other objects
#                 if classLabels[ClassInd - 1].lower() == 'person':
#                     print("Person")
#                     arduino.write(b'B')  # Send signal for buzzer
#                 else:
#                     arduino.write(b'V')  # Send signal for vibrator

#     # Get window size dynamically for full window display
#     cv2.namedWindow('Object Detection', cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty('Object Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
#     window_width, window_height = cv2.getWindowImageRect('Object Detection')[2], cv2.getWindowImageRect('Object Detection')[3]
    
#     # Resize the frame to the window size
#     display_frame = cv2.resize(frame, (window_width, window_height))

#     cv2.imshow('Object Detection', display_frame)
#     file_path = fr'C:\Users\Jaisriram\Downloads\detected_frame_{frame_num}.png'
#     cv2.imwrite(file_path, frame)

#     # Break loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release resources
# cv2.destroyAllWindows()
# arduino.close()


In [ ]:
import cv2
import serial
import time
import numpy as np

# Initialize serial communication with Arduino
arduino = serial.Serial('COM5', 700000, timeout=2)  # Timeout after 2 seconds
time.sleep(2)  # Wait for the connection to be established

# Load pre-trained SSD MobileNet model
config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# Initialize the detection model
model = cv2.dnn_DetectionModel(frozen_model, config_file)

# Load class labels
classLabels = []
file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')

# Set model parameters
model.setInputSize(320, 320)
model.setInputScale(1.0 / 127.5)
model.setInputMean((127.5, 127.5, 127.5))
model.setInputSwapRB(True)

# Define font and font scale for displaying labels
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5

# Image dimensions from the Arduino (320x240 for QVGA)
image_width = 320
image_height = 240

# Frame counter for unique filenames
frame_num = 0

# Function to convert RGB565 to RGB888
def rgb565_to_rgb888(data):
    r = (data >> 11) & 0x1F
    g = (data >> 5) & 0x3F
    b = data & 0x1F
    r = (r << 3) & 0xFF  # Scale 5-bit red to 8-bit
    g = (g << 2) & 0xFF  # Scale 6-bit green to 8-bit
    b = (b << 3) & 0xFF  # Scale 5-bit blue to 8-bit
    return r, g, b

# Read the image data from Arduino and process it
def read_image_from_arduino():
    image_data = []
    start_time = time.time()
    bytes_per_image = image_width * image_height * 2  # 2 bytes per pixel (RGB565)

    while len(image_data) < bytes_per_image:
        if arduino.in_waiting > 0:
            byte = arduino.read(1)
            image_data.append(ord(byte))
        elif time.time() - start_time > 2:  # Timeout after 2 seconds
            print("Timeout: No data received from Arduino")
            return None  # Return None if no data was received
    
    # Convert the raw data into RGB565 format
    image_rgb565 = np.array(image_data, dtype=np.uint8).reshape((image_height, image_width, 2))
    
    # Convert RGB565 to RGB888 for OpenCV
    image_rgb = np.zeros((image_height, image_width, 3), dtype=np.uint8)
    for y in range(image_height):
        for x in range(image_width):
            pixel_data = (image_rgb565[y, x, 0] << 8) | image_rgb565[y, x, 1]
            r, g, b = rgb565_to_rgb888(pixel_data)
            image_rgb[y, x] = [b, g, r]  # OpenCV uses BGR format

    # Resize the image to the input size expected by the model
    image_resized = cv2.resize(image_rgb, (320, 240))

    return image_resized

# Main loop for detection
while True:
    # Read the image from the Arduino
    frame = read_image_from_arduino()
#     frame = cap.read()
    
    if frame is None:
        continue  # Skip this loop iteration if no frame is received

    # Detect objects in the frame
    classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

    # Check if any objects are detected
    if len(classIndex) != 0:
        for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:  # Process only known COCO classes (1-80)
                # Draw bounding box around detected objects
                cv2.rectangle(frame, boxes, (0, 255, 0), 2)
                cv2.putText(frame, classLabels[ClassInd - 1], 
                            (boxes[0], boxes[1] - 10), font, 
                            fontScale=font_scale, color=(0, 255, 0), thickness=2)
                
                # Activate Arduino buzzer for 'car', vibrator for other objects
                if classLabels[ClassInd - 1].lower() == 'car':
                    print("car")
                    arduino.write(b'B')  # Send signal for buzzer
                elif classLabels[ClassInd - 1].lower() == 'person':
                    print("person")
                    arduino.write(b'V')  # Send signal for vibrator

    # Get window size dynamically for full window display
    cv2.namedWindow('Object Detection', cv2.WINDOW_NORMAL)
    cv2.setWindowProperty('Object Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    window_width, window_height = cv2.getWindowImageRect('Object Detection')[2], cv2.getWindowImageRect('Object Detection')[3]
    
    # Resize the frame to the window size
    display_frame = cv2.resize(frame, (window_width, window_height))

    # Display the resulting frame
    cv2.imshow('Object Detection', display_frame)

    # Save the detected frame as an image with a unique filename
    file_path = fr'C:\Users\Jaisriram\Downloads\data from cam\detected_frame_{frame_num}.png'
    cv2.imwrite(file_path, frame)
    
    # Increment frame counter for unique filename
    frame_num += 1

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cv2.destroyAllWindows()
arduino.close()


In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_curve, auc

# # Assuming you have the following data from the detection loop:
# # `y_true` - list of true labels (1 for car, 0 for non-car or missed car)
# # `y_scores` - list of detection confidence scores

# # Example (Replace these with your actual values from detection)
# # y_true = [1, 0, 1, 1, 0, 1, 0, 1]     # True labels
# # y_scores = [0.9, 0.4, 0.85, 0.78, 0.3, 0.95, 0.2, 0.8]  # Confidence scores

# # Calculate ROC Curve and AUC
# fpr, tpr, thresholds = roc_curve(y_true, y_scores)
# roc_auc = auc(fpr, tpr)

# # Plot ROC Curve
# plt.figure(figsize=(8, 6))
# plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) for Car Detection')
# plt.legend(loc="lower right")
# plt.grid(True)
# plt.show()

# # Calculate and print detection accuracy
# true_positives = y_true.count(1)  # Replace with your actual count of true positives
# false_positives = y_true.count(0)  # Replace with your actual count of false positives
# false_negatives = len(y_true) - true_positives - false_positives  # Calculate false negatives if needed

# total_detections = true_positives + false_positives + false_negatives
# accuracy = true_positives / total_detections if total_detections > 0 else 0
# print(f"Detection Accuracy: {accuracy:.2f}")


In [ ]:
# import cv2
# import serial
# import time
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_curve, auc

# # Initialize serial communication with Arduino
# arduino = serial.Serial('COM5', 700000, timeout=2)
# time.sleep(2)  # Wait for the connection to be established

# # Load pre-trained SSD MobileNet model
# config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
# frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# # Initialize the detection model
# model = cv2.dnn_DetectionModel(frozen_model, config_file)

# # Load class labels
# classLabels = []
# file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
# with open(file_name, 'rt') as fpt:
#     classLabels = fpt.read().rstrip('\n').split('\n')

# # Set model parameters
# model.setInputSize(320, 320)
# model.setInputScale(1.0 / 127.5)
# model.setInputMean((127.5, 127.5, 127.5))
# model.setInputSwapRB(True)

# # Initialize lists to store true labels and scores
# y_true = []
# y_scores = []

# # Function to convert RGB565 to RGB888
# def rgb565_to_rgb888(data):
#     r = (data >> 11) & 0x1F
#     g = (data >> 5) & 0x3F
#     b = data & 0x1F
#     r = (r << 3) & 0xFF
#     g = (g << 2) & 0xFF
#     b = (b << 3) & 0xFF
#     return r, g, b

# # Read the image data from Arduino
# def read_image_from_arduino():
#     # Implement your image reading logic here
#     # For example, read the image data from serial
#     # Return the processed image
#     pass  # Replace this with actual image reading code

# # Main loop for detection
# frame_count = 0  # Count processed frames
# max_frames = 100  # Set the number of frames to process for ROC
# frames_to_wait = 10  # Number of frames to wait before plotting

# while frame_count < max_frames:
#     # Read the image from the Arduino
#     frame = read_image_from_arduino()

#     if frame is None:
#         continue  # Skip this loop iteration if no frame is received

#     # Detect objects in the frame
#     classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

#     # Check if any objects are detected
#     if len(classIndex) != 0:
#         for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
#             if ClassInd <= 80:  # Process only known COCO classes (1-80)
#                 # Draw bounding box around detected objects
#                 cv2.rectangle(frame, boxes, (0, 255, 0), 2)
#                 cv2.putText(frame, classLabels[ClassInd - 1], 
#                             (boxes[0], boxes[1] - 10), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#                 # Collect true labels and confidence scores
#                 if classLabels[ClassInd - 1].lower() == 'car':
#                     y_true.append(1)  # True label for car detected
#                     y_scores.append(conf)  # Confidence score for car detection
#                 else:
#                     y_true.append(0)  # True label for non-car
#                     y_scores.append(conf)  # Confidence score for non-car

#     # Increment the frame count
#     frame_count += 1

#     # Display the frame
#     cv2.imshow('Object Detection', frame)

#     # Break loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release resources
# cv2.destroyAllWindows()
# arduino.close()

# # Plot ROC Curve and calculate AUC after exiting the loop
# if len(y_true) >= frames_to_wait:
#     fpr, tpr, thresholds = roc_curve(y_true, y_scores)
#     roc_auc = auc(fpr, tpr)

#     # Plot ROC Curve
#     plt.figure(figsize=(8, 6))
#     plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
#     plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic (ROC) for Car Detection')
#     plt.legend(loc="lower right")
#     plt.grid(True)
#     plt.show()

#     # Calculate and print detection accuracy
#     true_positives = y_true.count(1)  # Count of true positives
#     false_positives = y_true.count(0)  # Count of false positives
#     total_detections = len(y_true)
#     accuracy = true_positives / total_detections if total_detections > 0 else 0
#     print(f"Detection Accuracy: {accuracy:.2f}")
# else:
#     print("Not enough data to plot ROC curve.")


In [ ]:
# import cv2
# import serial
# import time
# import numpy as np

# # Initialize serial communication with Arduino
# arduino = serial.Serial('COM5', 700000, timeout=2)  # Timeout after 2 seconds
# time.sleep(2)  # Wait for the connection to be established

# # Load pre-trained SSD MobileNet model
# config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
# frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# # Initialize the detection model
# model = cv2.dnn_DetectionModel(frozen_model, config_file)

# # Load class labels
# classLabels = []
# file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
# with open(file_name, 'rt') as fpt:
#     classLabels = fpt.read().rstrip('\n').split('\n')

# # Set model parameters
# model.setInputSize(320, 320)
# model.setInputScale(1.0 / 127.5)
# model.setInputMean((127.5, 127.5, 127.5))
# model.setInputSwapRB(True)

# # Define font and font scale for displaying labels
# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 0.5

# # Image dimensions from the Arduino (320x240 for QVGA)
# image_width = 320
# image_height = 240

# # Function to convert RGB565 to RGB888 using NumPy
# def rgb565_to_rgb888_numpy(image_rgb565):
#     # Split 2-byte RGB565 data into separate channels
#     red = (image_rgb565[:, :, 0] >> 3) * 8  # Scale 5 bits to 8 bits
#     green = ((image_rgb565[:, :, 0] & 0x07) << 3) | (image_rgb565[:, :, 1] >> 5)
#     green *= 4  # Scale 6 bits to 8 bits
#     blue = (image_rgb565[:, :, 1] & 0x1F) * 8  # Scale 5 bits to 8 bits

#     # Stack them into an RGB image in BGR format for OpenCV
#     image_rgb = np.stack((blue, green, red), axis=-1).astype(np.uint8)
#     return image_rgb

# # Read the image data from Arduino and process it
# def read_image_from_arduino():
#     image_data = []
#     start_time = time.time()
#     bytes_per_image = image_width * image_height * 2  # 2 bytes per pixel (RGB565)

#     while len(image_data) < bytes_per_image:
#         if arduino.in_waiting > 0:
#             byte = arduino.read(1)
#             image_data.append(ord(byte))
#         elif time.time() - start_time > 2:  # Timeout after 2 seconds
#             print("Timeout: No data received from Arduino")
#             return None  # Return None if no data was received
    
#     # Convert the raw data into RGB565 format
#     image_rgb565 = np.array(image_data, dtype=np.uint8).reshape((image_height, image_width, 2))
    
#     # Convert RGB565 to RGB888 using the NumPy function
#     image_rgb = rgb565_to_rgb888_numpy(image_rgb565)

#     # Resize the image to the input size expected by the model
#     image_resized = cv2.resize(image_rgb, (320, 240))

#     return image_resized

# # Main loop for detection
# while True:
#     # Read the image from the Arduino
#     frame = read_image_from_arduino()
    
#     if frame is None:
#         continue  # Skip this loop iteration if no frame is received

#     # Detect objects in the frame
#     classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

#     # Check if any objects are detected
#     if len(classIndex) != 0:
#         for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
#             if ClassInd <= 80:  # Process only known COCO classes (1-80)
#                 # Draw bounding box around detected objects
#                 cv2.rectangle(frame, boxes, (0, 255, 0), 2)
#                 cv2.putText(frame, classLabels[ClassInd - 1], 
#                             (boxes[0], boxes[1] - 10), font, 
#                             fontScale=font_scale, color=(0, 255, 0), thickness=2)
                
#                 # Activate Arduino buzzer for 'car', vibrator for other objects
#                 if classLabels[ClassInd - 1].lower() == 'person':
#                     print("Person")
#                     arduino.write(b'B')  # Send signal for buzzer
#                 else:
#                     arduino.write(b'V')  # Send signal for vibrator

#     # Get window size dynamically for full window display
#     cv2.namedWindow('Object Detection', cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty('Object Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
#     window_width, window_height = cv2.getWindowImageRect('Object Detection')[2], cv2.getWindowImageRect('Object Detection')[3]
    
#     # Resize the frame to the window size
#     display_frame = cv2.resize(frame, (window_width, window_height))

#     cv2.imshow('Object Detection', display_frame)

#     # Break loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release resources
# cv2.destroyAllWindows()
# arduino.close()


In [ ]:
# !pip uninstall pyserial
# !pip install pyserial
# 

In [ ]:
# import cv2
# import serial
# import time
# import numpy as np

# # Initialize serial communication with Arduino
# arduino = serial.Serial('COM5', 500000, timeout=2)  # Timeout after 2 seconds
# time.sleep(2)  # Wait for the connection to be established

# # Load pre-trained SSD MobileNet model
# config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
# frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# # Initialize the detection model
# model = cv2.dnn_DetectionModel(frozen_model, config_file)

# # Load class labels
# classLabels = []
# file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
# with open(file_name, 'rt') as fpt:
#     classLabels = fpt.read().rstrip('\n').split('\n')

# # Set model parameters
# # Reduce the input size to 240x240 for better performance
# model.setInputSize(240, 240)
# model.setInputScale(1.0 / 127.5)
# model.setInputMean((127.5, 127.5, 127.5))
# model.setInputSwapRB(True)

# # Define font and font scale for displaying labels
# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 0.5

# # Image dimensions from the Arduino (320x240 for QVGA)
# image_width = 320
# image_height = 240

# # Function to convert RGB565 to RGB888
# def rgb565_to_rgb888(data):
#     r = (data >> 11) & 0x1F
#     g = (data >> 5) & 0x3F
#     b = data & 0x1F
#     r = (r * 255) // 31  # Scale 5-bit red to 8-bit
#     g = (g * 255) // 63  # Scale 6-bit green to 8-bit
#     b = (b * 255) // 31  # Scale 5-bit blue to 8-bit
#     return r, g, b

# # Read the image data from Arduino and process it
# def read_image_from_arduino():
#     image_data = []
#     start_time = time.time()
#     bytes_per_image = image_width * image_height * 2  # 2 bytes per pixel (RGB565)

#     while len(image_data) < bytes_per_image:
#         if arduino.in_waiting > 0:
#             byte = arduino.read(1)
#             image_data.append(ord(byte))
#         elif time.time() - start_time > 2:  # Timeout after 2 seconds
#             print("Timeout: No data received from Arduino")
#             return None  # Return None if no data was received
    
#     # Convert the raw data into RGB565 format
#     image_rgb565 = np.array(image_data, dtype=np.uint8).reshape((image_height, image_width, 2))
    
#     # Convert RGB565 to RGB888 for OpenCV
#     image_rgb = np.zeros((image_height, image_width, 3), dtype=np.uint8)
#     for y in range(image_height):
#         for x in range(image_width):
#             # Ensure correct byte order (MSB first)
#             pixel_data = (image_rgb565[y, x, 0] << 8) | image_rgb565[y, x, 1]
#             r, g, b = rgb565_to_rgb888(pixel_data)
#             image_rgb[y, x] = [b, g, r]  # OpenCV uses BGR format

#     # Resize the image to reduce processing load
#     image_resized = cv2.resize(image_rgb, (160, 120))  # Downscaling the resolution

#     return image_resized

# # Main loop for detection
# frame_skip_counter = 0  # Counter for skipping frames

# while True:
#     # Read the image from the Arduino
#     frame = read_image_from_arduino()
    
#     if frame is None:
#         continue  # Skip this loop iteration if no frame is received

#     # Skip frames to reduce processing load (only detect every 5 frames)
#     frame_skip_counter += 1
#     if frame_skip_counter % 5 != 0:
#         continue

#     # Detect objects in the frame
#     classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

#     # Check if any objects are detected
#     if len(classIndex) != 0:
#         for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
#             if ClassInd <= 80:  # Process only known COCO classes (1-80)
#                 # Draw bounding box around detected objects
#                 cv2.rectangle(frame, boxes, (0, 255, 0), 2)
#                 cv2.putText(frame, classLabels[ClassInd - 1], 
#                             (boxes[0], boxes[1] - 10), font, 
#                             fontScale=font_scale, color=(0, 255, 0), thickness=2)
                
#                 # Activate Arduino buzzer for 'car', vibrator for other objects
#                 if classLabels[ClassInd - 1].lower() == 'person':
#                     print("Vehicle detected")
#                     arduino.write(b'B')  # Send signal for buzzer
#                 else:
#                     arduino.write(b'V')  # Send signal for vibrator

#     # Display the frame without resizing for now (downscaled resolution)
#     cv2.imshow('Object Detection', frame)

#     # Break loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release resources
# cv2.destroyAllWindows()
# arduino.close()


In [ ]:
# import cv2
# import serial
# import time
# import numpy as np

# # Initialize serial communication with Arduino
# arduino = serial.Serial('COM5', 500000, timeout=2)  # Timeout after 2 seconds
# time.sleep(2)  # Wait for the connection to be established

# # Load pre-trained SSD MobileNet model
# config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
# frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# # Initialize the detection model
# model = cv2.dnn_DetectionModel(frozen_model, config_file)

# # Load class labels
# classLabels = []
# file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
# with open(file_name, 'rt') as fpt:
#     classLabels = fpt.read().rstrip('\n').split('\n')

# # Set model parameters
# model.setInputSize(320, 320)
# model.setInputScale(1.0 / 127.5)
# model.setInputMean((127.5, 127.5, 127.5))
# model.setInputSwapRB(True)

# # Define font and font scale for displaying labels
# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 0.5

# # Image dimensions from the Arduino (320x240 for QVGA)
# image_width = 320
# image_height = 240

# # Function to convert RGB565 to RGB888
# def rgb565_to_rgb888(data):
#     r = (data >> 11) & 0x1F
#     g = (data >> 5) & 0x3F
#     b = data & 0x1F
#     r = (r << 3) & 0xFF  # Scale 5-bit red to 8-bit
#     g = (g << 2) & 0xFF  # Scale 6-bit green to 8-bit
#     b = (b << 3) & 0xFF  # Scale 5-bit blue to 8-bit
#     return r, g, b

# def read_image_from_arduino(): 
#     image_data = []
#     start_time = time.time()
#     bytes_per_image = image_width * image_height * 2  # 2 bytes per pixel (RGB565)

#     while len(image_data) < bytes_per_image:
#         if arduino.in_waiting > 0:
#             byte = arduino.read(1)
#             image_data.append(ord(byte))
#         elif time.time() - start_time > 2:  # Timeout after 2 seconds
#             print("Timeout: No data received from Arduino")
#             return None  # Return None if no data was received
    
#     # Convert the raw data into RGB565 format
#     image_rgb565 = np.array(image_data, dtype=np.uint8).reshape((image_height, image_width, 2))
    
#     # Convert RGB565 to RGB888 for OpenCV
#     image_rgb = np.zeros((image_height, image_width, 3), dtype=np.uint8)
#     for y in range(image_height):
#         for x in range(image_width):
#             pixel_data = (image_rgb565[y, x, 0] << 8) | image_rgb565[y, x, 1]
#             r, g, b = rgb565_to_rgb888(pixel_data)
#             image_rgb[y, x] = [b, g, r]  # OpenCV uses BGR format

#     # Resize the image to the input size expected by the model
#     image_resized = cv2.resize(image_rgb, (320, 240))

#     return image_resized

# # Main loop for detection
# while True:
#     # Read the image from the Arduino
#     frame = read_image_from_arduino()
    
#     if frame is None:
#         continue  # Skip this loop iteration if no frame is received

#     # Detect objects in the frame
#     classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

#     # Check if any objects are detected
#     if len(classIndex) != 0:
#         for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
#             if ClassInd <= 80:  # Process only known COCO classes (1-80)
#                 # Draw bounding box around detected objects
#                 cv2.rectangle(frame, boxes, (0, 255, 0), 2)
#                 cv2.putText(frame, classLabels[ClassInd - 1], 
#                             (boxes[0], boxes[1] - 10), font, 
#                             fontScale=font_scale, color=(0, 255, 0), thickness=2)
                
#                 # Activate Arduino buzzer for 'car', vibrator for other objects
#                 if classLabels[ClassInd - 1].lower() == 'person':
#                     print("Vehicle detected")
#                     arduino.write(b'B')  # Send signal for buzzer
#                 else:
#                     arduino.write(b'V')  # Send signal for vibrator

#     # Compress the frame using JPEG encoding with a lower quality
#     encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50]  # 50% quality for compression
#     result, compressed_image = cv2.imencode('.jpg', frame, encode_param)

#     if result:
#         # Decode the compressed image back for display
#         frame = cv2.imdecode(compressed_image, cv2.IMREAD_COLOR)
    
#     # Get window size dynamically for full window display
#     cv2.namedWindow('Object Detection', cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty('Object Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
#     window_width, window_height = cv2.getWindowImageRect('Object Detection')[2], cv2.getWindowImageRect('Object Detection')[3]
    
#     # Resize the frame to the window size
#     display_frame = cv2.resize(frame, (window_width, window_height))

#     cv2.imshow('Object Detection', display_frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cv2.destroyAllWindows()


In [ ]:
# pip install JPype1


In [ ]:
# import cv2
# import serial
# import time
# import numpy as np

# # Initialize serial communication with Arduino
# arduino = serial.Serial('COM5', 500000, timeout=2)  # Timeout after 2 seconds
# time.sleep(2)  # Wait for the connection to be established

# # Load pre-trained SSD MobileNet model
# config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
# frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# # Initialize the detection model
# model = cv2.dnn_DetectionModel(frozen_model, config_file)

# # Load class labels
# classLabels = []
# file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
# with open(file_name, 'rt') as fpt:
#     classLabels = fpt.read().rstrip('\n').split('\n')

# # Set model parameters
# model.setInputSize(320, 320)
# model.setInputScale(1.0 / 127.5)
# model.setInputMean((127.5, 127.5, 127.5))
# model.setInputSwapRB(True)

# # Define font and font scale for displaying labels
# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 0.5

# # Image dimensions from the Arduino (adjust as per your camera resolution)
# image_width = 120  # QQVGA width
# image_height = 160  # QQVGA height

# # Function to convert RGB565 to RGB888
# def rgb565_to_rgb888(data):
#     r = (data >> 11) & 0x1F
#     g = (data >> 5) & 0x3F
#     b = data & 0x1F
#     r = (r << 3) & 0xFF  # Scale 5-bit red to 8-bit
#     g = (g << 2) & 0xFF  # Scale 6-bit green to 8-bit
#     b = (b << 3) & 0xFF  # Scale 5-bit blue to 8-bit
#     return r, g, b


# # Read the image data from Arduino and process it
# def read_image_from_arduino():
#     image_data = []
#     start_time = time.time()
#     bytes_per_image = image_width * image_height * 2  # 2 bytes per pixel (RGB565)

#     image_data = []
#     bytes_per_image = 160 * 120 * 2  # 38400 bytes expected

#     # Read the image data from Arduino
#     while len(image_data) < bytes_per_image:
#         if arduino.in_waiting > 0:
#             byte = arduino.read(1)
#             image_data.append(ord(byte))
#         elif time.time() - start_time > 2:  # Timeout
#             print("Timeout: No data received from Arduino")
#             return None

#     # Verify the length of received data
#     if len(image_data) != bytes_per_image:
#         print(f"Error: Expected {bytes_per_image} bytes, but received {len(image_data)} bytes.")

    
#     # Convert the raw data into RGB565 format
#     image_rgb565 = np.array(image_data, dtype=np.uint8).reshape((image_height, image_width, 2))
    
#     # Convert RGB565 to RGB888 for OpenCV
#     image_rgb = np.zeros((image_height, image_width, 3), dtype=np.uint8)
#     for y in range(image_height):
#         for x in range(image_width):
#             pixel_data = (image_rgb565[y, x, 0] << 8) | image_rgb565[y, x, 1]
#             r, g, b = rgb565_to_rgb888(pixel_data)
#             image_rgb[y, x] = [b, g, r]  # OpenCV uses BGR format
#     image_resized = cv2.resize(image_rgb, (320, 320))

#     return image_resized  # Return the RGB image directly without resizing

# # Main loop for detection
# # while True:
# # Read the image from the Arduino
# frame = read_image_from_arduino()

# # if frame is None:
# #     continue  # Skip this loop iteration if no frame is received

# # Detect objects in the frame
# classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

# # Check if any objects are detected
# if len(classIndex) != 0:
#     for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
#         if ClassInd <= 80:  # Process only known COCO classes (1-80)
#             # Draw bounding box around detected objects
#             cv2.rectangle(frame, boxes, (0, 255, 0), 2)
#             cv2.putText(frame, classLabels[ClassInd - 1], 
#                         (boxes[0], boxes[1] - 10), font, 
#                         fontScale=font_scale, color=(0, 255, 0), thickness=2)

#             # Activate Arduino buzzer for 'car', vibrator for other objects
#             if classLabels[ClassInd - 1].lower() == 'car':
#                 print("Vehicle detected")
#                 arduino.write(b'B')  # Send signal for buzzer
#             else:
#                 arduino.write(b'V')  # Send signal for vibrator

#     # Get window size dynamically for full window display
#     cv2.namedWindow('Object Detection', cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty('Object Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
#     # Display the frame without resizing
#     cv2.imshow('Object Detection', frame)

#     # Break loop on 'q' key press
# #     if cv2.waitKey(1) & 0xFF == ord('q'):
# #         break

# # Release resources
# # cv2.destroyAllWindows()
# # arduino.close()


In [ ]:
# import cv2
# import serial
# import time
# import numpy as np

# # Initialize serial communication with Arduino
# arduino = serial.Serial('COM5', 500000, timeout=2)  # Timeout after 2 seconds
# time.sleep(2)  # Wait for the connection to be established

# # Load pre-trained SSD MobileNet model
# config_file = r'C:\Users\Jaisriram\Downloads\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
# frozen_model = r'C:\Users\Jaisriram\Downloads\frozen_inference_graph.pb'

# # Initialize the detection model
# model = cv2.dnn_DetectionModel(frozen_model, config_file)

# # Load class labels
# classLabels = []
# file_name = r'C:\Users\Jaisriram\Downloads\labels.txt'
# with open(file_name, 'rt') as fpt:
#     classLabels = fpt.read().rstrip('\n').split('\n')

# # Set model parameters
# model.setInputSize(320, 320)
# model.setInputScale(1.0 / 127.5)
# model.setInputMean((127.5, 127.5, 127.5))
# model.setInputSwapRB(True)

# # Define font and font scale for displaying labels
# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 0.5

# # Image dimensions from the Arduino (adjust as per your camera resolution)
# image_width = 120  # QQVGA width
# image_height = 160  # QQVGA height

# # Function to convert RGB565 to RGB888
# def rgb565_to_rgb888(data):
#     r = (data >> 11) & 0x1F
#     g = (data >> 5) & 0x3F
#     b = data & 0x1F
#     r = (r << 3) & 0xFF  # Scale 5-bit red to 8-bit
#     g = (g << 2) & 0xFF  # Scale 6-bit green to 8-bit
#     b = (b << 3) & 0xFF  # Scale 5-bit blue to 8-bit
#     return r, g, b

# # Read the image data from Arduino and process it
# def read_image_from_arduino():
#     image_data = []
#     start_time = time.time()
#     bytes_per_image = image_width * image_height * 2  # 2 bytes per pixel (RGB565)

#     image_data = []
#     bytes_per_image = 160 * 120 * 2  # 38400 bytes expected

#     # Read the image data from Arduino
#     while len(image_data) < bytes_per_image:
#         if arduino.in_waiting > 0:
# #             time.sleep(10)
#             byte = arduino.read(1)
#             image_data.append(ord(byte))
#         elif time.time() - start_time > 2:  # Timeout
#             print("Timeout: No data received from Arduino")
#             return None

#     # Verify the length of received data
#     if len(image_data) != bytes_per_image:
#         print(f"Error: Expected {bytes_per_image} bytes, but received {len(image_data)} bytes.")

    
#     # Convert the raw data into RGB565 format
#     image_rgb565 = np.array(image_data, dtype=np.uint8).reshape((image_height, image_width, 2))
    
#     # Convert RGB565 to RGB888 for OpenCV
#     image_rgb = np.zeros((image_height, image_width, 3), dtype=np.uint8)
#     for y in range(image_height):
#         for x in range(image_width):
#             pixel_data = (image_rgb565[y, x, 0] << 8) | image_rgb565[y, x, 1]
#             r, g, b = rgb565_to_rgb888(pixel_data)
#             image_rgb[y, x] = [b, g, r]  # OpenCV uses BGR format
#     image_resized = cv2.resize(image_rgb, (320, 320))

#     return image_resized  # Return the RGB image directly without resizing

# # Main loop to capture 5 frames
# num_frames = 30
# for frame_num in range(1, num_frames + 1):
#     print(f"Capturing frame {frame_num}...")
#     time.sleep(2)  # Delay before capturing each frame

#     frame = read_image_from_arduino()

#     # Detect objects in the frame
#     classIndex, confidence, bbox = model.detect(frame, confThreshold=0.5)

#     # Check if any objects are detected
#     if len(classIndex) != 0:
#         for ClassInd, conf, boxes in zip(classIndex.flatten(), confidence.flatten(), bbox):
#             if ClassInd <= 80:  # Process only known COCO classes (1-80)
#                 # Draw bounding box around detected objects
#                 cv2.rectangle(frame, boxes, (0, 255, 0), 2)
#                 cv2.putText(frame, classLabels[ClassInd - 1], 
#                             (boxes[0], boxes[1] - 10), font, 
#                             fontScale=font_scale, color=(0, 255, 0), thickness=2)

#                 # Activate Arduino buzzer for 'car', vibrator for other objects
#                 if classLabels[ClassInd - 1].lower() == 'car':
#                     print("Vehicle detected")
#                     arduino.write(b'B')  # Send signal for buzzer
#                 else:
#                     arduino.write(b'V')  # Send signal for vibrator

#     # Display the frame in a window
#     cv2.imshow(f'Object Detection - Frame {frame_num}', frame)

#     # Save the frame as an image file
#     file_path = fr'C:\Users\Jaisriram\Downloads\detected_frame_{frame_num}.png'
#     cv2.imwrite(file_path, frame)
#     print(f"Frame {frame_num} saved as 'detected_frame_{frame_num}.png'")

#     # Wait for a short moment between frames
#     cv2.waitKey(1000)  # Display each frame for 1 second before moving to the next

# # Close the windows
# cv2.destroyAllWindows()

# # Close the serial connection
# arduino.close()


In [ ]:
# import serial
# import numpy as np
# import matplotlib.pyplot as plt

# # Configure the serial port
# port = 'COM5'  # Change this to your port
# baudrate = 50000  # Make sure this matches the baud rate in your microcontroller

# # Open the serial connection
# ser = serial.Serial(port, baudrate, timeout=1)

# def read_image_from_serial(ser, width, height):
#     # Wait for the start marker
#     start_marker = None
#     while start_marker != b'\xFF':
#         start_marker = ser.read()  # Read one byte at a time
#         if start_marker:
#             print(f"Received byte: {start_marker.hex()}")  # Print received byte for debugging

#     print("Start marker found")

#     # Read pixel data
#     pixel_data = bytearray()

#     while True:
#         byte = ser.read()
#         if byte == b'\xFE':  # End marker
#             break
#         pixel_data.append(byte[0])  # Append the byte to the pixel data

#     # Debug: Check how many bytes were read
#     total_bytes_read = len(pixel_data)
#     print(f"Total bytes read: {total_bytes_read}")

#     # Validate the number of bytes received
#     expected_size = width * height * 3  # 3 bytes per pixel for RGB
#     if total_bytes_read != expected_size:
#         print(f"Warning: Expected {expected_size} bytes, but received {total_bytes_read} bytes.")
#         return None  # Return None or handle error appropriately

#     # Convert bytearray to numpy array for landscape orientation
#     # Reshape for RGB (height, width, channels)
#     image_array = np.frombuffer(pixel_data, dtype=np.uint8).reshape((height, width, 3))

#     return image_array


# # Assuming the image resolution (width, height) is known
# image_width = 160  # Set according to your camera's output
# image_height = 180  # Set according to your camera's output

# # Read the image data from serial
# image_array = read_image_from_serial(ser, image_width, image_height)

# # Close the serial connection
# ser.close()

# # Display the image
# if image_array is not None:
#     plt.imshow(image_array, cmap='gray')  # Assuming the image is grayscale
#     plt.axis('off')
#     plt.show()
